In [6]:
import pandas as pd


In [9]:
z1=pd.read_parquet("D:\OneDrive - Northwestern University\My files\data_feeds\zacks\data\zacks_zern_eadates.parquet",columns=['zid','ea_date'])
z2=pd.read_parquet("D:\OneDrive - Northwestern University\My files\data_feeds\zacks\data\wrds_zacks_sales_eadates.parquet",columns=['zid','ea_date'])
z3=pd.read_parquet("D:\OneDrive - Northwestern University\My files\data_feeds\zacks\data\wrds_zacks_eps_eadates.parquet",columns=['zid','ea_date'])

zacks=pd.concat([z1,z2,z3])
zacks



,zid,ea_date
0,Z86Z,1993-08-26
1,Z86Z,1993-12-02
2,Z86Z,1994-03-03
3,Z86Z,1994-05-26
4,Z86Z,1994-08-25
...,...,...
205703,ZZ,2011-09-27
205704,ZZ,2012-01-18
205705,ZZ,2012-03-27
205706,ZZ,2012-06-26


In [10]:
zacks=zacks.drop_duplicates().reset_index(drop=True)
zacks
#now merge with zacks names to get the hticker and cusips
#unfortunately becasue pandas kinda sucks, we need to use sqlite3


,zid,ea_date
0,Z86Z,1993-08-26
1,Z86Z,1993-12-02
2,Z86Z,1994-03-03
3,Z86Z,1994-05-26
4,Z86Z,1994-08-25
...,...,...
794393,ZUES,1995-07-25
794394,ZUES,1995-10-25
794395,ZY,1995-03-01
794396,ZY,1995-05-16


In [14]:
import sqlite3
con = sqlite3.connect("D:\OneDrive - Northwestern University\My files\data_feeds\linking_databases\data\sec_names.db")


In [15]:
zacks.to_sql("zacks_ea_dates", con, index=False, if_exists='replace')

794398

In [23]:
sql1 = """SELECT a.*,b.ea_date 
FROM zacks_names  a inner join zacks_ea_dates as b
on (a.zid=b.zid )
AND ( a.start_date<= b.ea_date and  a.end_date>= b.ea_date)

order  by a.zid,b.ea_date;"""
zacks_matched = pd.read_sql_query(sql1, con)
zacks_matched

zid           object
hticker       object
hcusip        object
start_date    object
end_date      object
cusip8        object
root          object
suffix        object
tsym          object
crsp_tsym     object
bbg_tsym      object
ea_date       object
dtype: object

In [12]:
ibes=pd.read_parquet('D:\OneDrive - Northwestern University\My files\data_feeds\IBES\Data\ibes_eadates.parquet',columns=['ibes_id','ea_date'])
ibes

,ibes_id,ea_date
0,0000,2014-02-14
1,0000,2014-02-14
2,0000,2014-02-14
3,0000,2014-02-14
4,0000,2014-02-14
...,...,...
204467,ZYNX,2021-04-29
204468,ZYNX,2021-07-29
204469,ZYNX,2021-11-02
204470,ZYNX,2022-02-24


In [13]:
ibes=ibes.drop_duplicates().reset_index(drop=True)
ibes

,ibes_id,ea_date
0,0000,2014-02-14
1,0000,2014-05-06
2,0000,2014-08-06
3,0000,2014-11-04
4,0000,2015-01-30
...,...,...
835976,ZYNX,2021-02-25
835977,ZYNX,2021-04-29
835978,ZYNX,2021-07-29
835979,ZYNX,2021-11-02


In [19]:
ibes.to_sql("ibes_ea_dates", con, index=False, if_exists='replace')

835981

In [22]:
sql2 = """SELECT a.*,b.ea_date 
FROM ibes_names  a inner join ibes_ea_dates as b
on (a.ibes_id=b.ibes_id )
AND ( a.start_date<= b.ea_date and  a.end_date>= b.ea_date)

order  by a.ibes_id,b.ea_date;"""
ibes_matched = pd.read_sql_query(sql2, con)
ibes_matched

,ibes_id,cusip8,ticker,start_date,end_date,root,suffix,tsym,crsp_tsym,bbg_tsym,ea_date
0,0000,87482X10,TLMR,2014-02-20,2016-09-02,TLMR,,TLMR,TLMR,TLMR,2014-05-06 00:00:00
1,0000,87482X10,TLMR,2014-02-20,2016-09-02,TLMR,,TLMR,TLMR,TLMR,2014-08-06 00:00:00
2,0000,87482X10,TLMR,2014-02-20,2016-09-02,TLMR,,TLMR,TLMR,TLMR,2014-11-04 00:00:00
3,0000,87482X10,TLMR,2014-02-20,2016-09-02,TLMR,,TLMR,TLMR,TLMR,2015-01-30 00:00:00
4,0000,87482X10,TLMR,2014-02-20,2016-09-02,TLMR,,TLMR,TLMR,TLMR,2015-04-30 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...
812860,ZYNX,98986M10,ZYXI,2008-07-17,2021-11-03,ZYXI,,ZYXI,ZYXI,ZYXI,2020-10-27 00:00:00
812861,ZYNX,98986M10,ZYXI,2008-07-17,2021-11-03,ZYXI,,ZYXI,ZYXI,ZYXI,2021-02-25 00:00:00
812862,ZYNX,98986M10,ZYXI,2008-07-17,2021-11-03,ZYXI,,ZYXI,ZYXI,ZYXI,2021-04-29 00:00:00
812863,ZYNX,98986M10,ZYXI,2008-07-17,2021-11-03,ZYXI,,ZYXI,ZYXI,ZYXI,2021-07-29 00:00:00


In [28]:
cols=['tsym','cusip8','crsp_tsym','bbg_tsym','ea_date']
z=zacks_matched[cols]
i=ibes_matched[cols]
eas=pd.concat([z,i])
eas


,tsym,cusip8,crsp_tsym,bbg_tsym,ea_date
0,AAON,00036020,AAON,AAON,1993-10-12 00:00:00
1,AAON,00036020,AAON,AAON,1994-02-22 00:00:00
2,AAON,00036020,AAON,AAON,1994-04-12 00:00:00
3,AAON,00036020,AAON,AAON,1994-07-12 00:00:00
4,AAON,00036020,AAON,AAON,1994-10-11 00:00:00
...,...,...,...,...,...
812860,ZYXI,98986M10,ZYXI,ZYXI,2020-10-27 00:00:00
812861,ZYXI,98986M10,ZYXI,ZYXI,2021-02-25 00:00:00
812862,ZYXI,98986M10,ZYXI,ZYXI,2021-04-29 00:00:00
812863,ZYXI,98986M10,ZYXI,ZYXI,2021-07-29 00:00:00


In [29]:
eas=eas.drop_duplicates().reset_index(drop=True)
eas

,tsym,cusip8,crsp_tsym,bbg_tsym,ea_date
0,AAON,00036020,AAON,AAON,1993-10-12 00:00:00
1,AAON,00036020,AAON,AAON,1994-02-22 00:00:00
2,AAON,00036020,AAON,AAON,1994-04-12 00:00:00
3,AAON,00036020,AAON,AAON,1994-07-12 00:00:00
4,AAON,00036020,AAON,AAON,1994-10-11 00:00:00
...,...,...,...,...,...
1083032,ZYXI,98986M10,ZYXI,ZYXI,2010-05-17 00:00:00
1083033,ZYXI,98986M10,ZYXI,ZYXI,2010-08-16 00:00:00
1083034,ZYXI,98986M10,ZYXI,ZYXI,2010-11-12 00:00:00
1083035,ZYXI,98986M10,ZYXI,ZYXI,2015-03-31 00:00:00
